In [30]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
from torch.utils.data import Dataset, DataLoader
from torch import nn
from torch.nn.functional import relu
from scipy.io import loadmat
import os
from os import listdir
import pandas as pd
from skimage import io
from PIL import Image
from sklearn.metrics import confusion_matrix
import seaborn as sn
import pdb
import torch.nn.functional as F

from utils import *

In [31]:
val_dates = ['180328','180329']
test_dates = ['180330','180331']

In [32]:
train_loader, val_loader, test_loader = create_dataloaders(val_dates=val_dates, test_dates=test_dates, batch_size=128, version='_Goose_1st_1', upsample=True)

In [33]:
len(train_loader), len(val_loader), len(test_loader)

(258, 22, 51)

In [37]:
class CNN_small(nn.Module):
    def __init__(self):
        super(CNN_small,self).__init__()
         
        #######################    
        # convolutional layers
        # ^^^^^^^^^^^^^^^^^^^^
    
        # convolutional layer (sees 1x100x10 image tensor)
        self.conv1 = nn.Conv2d(in_channels=62, out_channels=2,kernel_size=(3,3),stride=(1,1),padding=(1,1))
        # convolutional layer (sees 2x50x5 tensor)
        self.conv2 = nn.Conv2d(in_channels=2, out_channels=4,kernel_size=(3,3),stride=(1,1),padding=(1,1))
        
        # Max pooling
        self.pool = nn.MaxPool2d(2,2)
        
        # dropout layer 
        self.dropout = nn.Dropout(0.2)
        
         # batch normalization 
        self.bn1 = nn.BatchNorm2d(2)
        self.bn2 = nn.BatchNorm2d(4)
        
        
        # linear layers
        # sees a layer 4x25x6
        self.fc1 = nn.Linear(4*25*2,1)
        
        
    def forward(self,x):
        
             
        #########################
        # CONVOLUTIONAL LAYERS ##
        #########################
        
        # Convolution 1:
        print('x shape before conv 1: ',x.shape)
        pdb.set_trace()
        # x = self.pad_B(x) # padding right one step  -- comment out for 46x20 images
        # x = x [:,:,1:,:] 
        x = x.float()
        conv_x1 = self.conv1(x)
        x = self.pool(F.relu(self.bn1(conv_x1))) # Convolution 2: in short convolution + relu + pooling  
        
#         print('x shape conv 1,, after padding: ',x.shape)
#         pdb.set_trace()
        # plt.imshow(x[1,0,:,:].detach().cpu().numpy().transpose(),origin='lower')
        # plt.show()

        
        # plt.imshow(x[1,0,:,:].detach().cpu().numpy().transpose(),origin='lower')
        # plt.show()

        print('x shape before conv 2: ',x.shape)
        pdb.set_trace()
        conv_x2 = self.conv2(x)
        x = self.pool(F.relu(self.bn2(conv_x2))) # Convolution 2: in short convolution + relu + pooling  
        
        
        print('x shape before MLP: ',x.shape)
        pdb.set_trace()
        ########################
        ## LINEAR LAYERS, MLP ##
        ########################
        
        x = x.view(x.shape[0],-1) # flatten image input
        # print('x shape in MLP: ',x.shape)
        # pdb.set_trace() 
        x = self.dropout(x) # dropout
        
        print('x shape before fc1: ',x.shape)
        pdb.set_trace() 
        # x = F.leaky_relu(self.fc1(x)) # Fully connected 1: linear + relu
        # x = self.dropout(x) # dropout
        
        x = self.fc1(x) # Fully connected 3: linear 
#         x = F.sigmoid(x)
        
        # print('x shape before fc2: ',x.shape)
        # pdb.set_trace() 

        return x, conv_x1, conv_x2 
        

In [38]:
device = torch.device('cuda')

tol = 0.01
learning_rate = 0.001
num_epochs = 51
alpha = 0
model_type = 'MLP'
loss_type = 'bce'
reg_type = 'none'
CH = 'all'

model = CNN_small().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)

In [39]:
epochs, training_losses, training_acc, val_losses, validation_acc = [], [], [], [], []

for epoch in range(num_epochs):
    train_loss, train_acc = train(model, optimizer, train_loader, alpha=alpha, model_type=model_type, loss_type=loss_type, reg_type=reg_type, collect_result=False, device=device)
    print('Train loss for epoch {}: {}'.format(epoch, train_loss))
    val_loss, val_acc = evaluate(model, optimizer, val_loader, alpha=alpha, model_type=model_type, loss_type=loss_type, reg_type=reg_type, collect_result=False, device=device)
    print('Val loss for epoch {}: {}'.format(epoch, val_loss))
    if val_losses:
        if val_loss < min(val_losses):
            best_epoch = epoch
            print(epoch)
            path = '/mnt/pesaranlab/People/Capstone_students/Yue/model/model_Goose_1st_1_upsample'
            torch.save(model.state_dict(), '{}/{}_CH{}_LOSS{}_REG{}{}_EPOCH{}.pt'.format(path, model_type, CH, loss_type, reg_type, alpha, epoch))
    epochs.append(epoch)
    training_losses.append(train_loss)
    training_acc.append(train_acc)
    val_losses.append(val_loss)
    validation_acc.append(val_acc)
    
    if train_loss < tol:
        break

x shape before conv 1:  torch.Size([128, 62, 100, 10])
> <ipython-input-37-4bfaabb8ddba>(42)forward()
     40         # x = self.pad_B(x) # padding right one step  -- comment out for 46x20 images
     41         # x = x [:,:,1:,:]
---> 42         x = x.float()
     43         conv_x1 = self.conv1(x)
     44         x = self.pool(F.relu(self.bn1(conv_x1))) # Convolution 2: in short convolution + relu + pooling

ipdb> continue
x shape before conv 2:  torch.Size([128, 2, 50, 5])
> <ipython-input-37-4bfaabb8ddba>(57)forward()
     55         print('x shape before conv 2: ',x.shape)
     56         pdb.set_trace()
---> 57         conv_x2 = self.conv2(x)
     58         x = self.pool(F.relu(self.bn2(conv_x2))) # Convolution 2: in short convolution + relu + pooling
     59 

ipdb> continue
x shape before MLP:  torch.Size([128, 4, 25, 2])
> <ipython-input-37-4bfaabb8ddba>(67)forward()
     65         ########################
     66 
---> 67         x = x.view(x.shape[0],-1) # flatten image in

AttributeError: 'tuple' object has no attribute 'reshape'